In [3]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 215)
from time import sleep

In [1]:
urls = ["http://www.sis.itu.edu.tr/eng/curriculums/plan/MAT/201410.html", "http://www.sis.itu.edu.tr/eng/curriculums/plan/MAT/20141011.html", "http://www.sis.itu.edu.tr/eng/curriculums/plan/MAT/20141051.html", "http://www.sis.itu.edu.tr/eng/curriculums/plan/MAT/20141011.html", "http://www.sis.itu.edu.tr/eng/curriculums/plan/MAT/20141021.html", "http://www.sis.itu.edu.tr/eng/curriculums/plan/MAT/20141081.html"]
curriculum = []
for url in urls:
    page = requests.get(url)
    sleep(10)
    soup = BeautifulSoup(page.content, 'html.parser')  
    semesters = soup.find_all('table', 'plan')
    for semester in semesters:
        rows = semester.find_all('tr')
        for idx,row in enumerate(rows):
            if idx != 0:
                values = row.find_all('td')
                code = values[0].text.strip()
                if not code:
                    continue
                name = values[1].text.strip()
                credit = values[2].text.strip()
                data = {"code": code, "name": name, "credit": credit}
                curriculum.append(data)
            
curriculum_df = pd.DataFrame(data = curriculum).set_index('code')         

# curriculum_df.to_csv("curriculum.csv")
            

In [37]:
curriculum_df = pd.read_csv("curriculum.csv", index_col= "code")
curriculum_df.head()

,name,credit
code,,
KIM 101E,General Chemistry I,3.0
KIM 101EL,General Chemistry I Laboratory,1.0
BIL 101E,Intro. to Computer and Inf.Systems,1.5
MAT 111,Mathematics I,5.0
FIZ 101E,Physics I,3.0


In [38]:
codes_full = pd.Series(curriculum_df.index)
codes_short = codes_full.apply(lambda x: x[:3]).unique()
codes_short[:10]

array(['KIM', 'BIL', 'MAT', 'FIZ', 'MUH', 'TUR', 'DNK', 'ING', 'ATA',
       'EKO'], dtype=object)

In [7]:
schedule = []
short_url = "http://www.sis.itu.edu.tr/tr/ders_programlari/LSprogramlar/prg.php?fb="
for code in codes_short:
    full_url = short_url + code
    page = requests.get(full_url)
    sleep(10)
    soup = BeautifulSoup(page.content, 'html.parser')
    rows = soup.find_all('tr', {'bgcolor': '#FFFFFF'})

    for row in rows:
        values = row.find_all('td')
        code = values[1].text
        if code not in codes_full.values:  
            continue
        else:
            crn = values[0].text
            day = values[5].text
            time = values[6].text.split('/')
            starts = time[0]
            ends = time[1]
            schedule.append({"crn": crn, "code": code, "day": day, "starts": starts, "ends": ends})

schedule_df = pd.DataFrame(data = schedule).set_index('crn')         

# schedule_df.to_csv("schedule.csv")

In [39]:
schedule_df = pd.read_csv("schedule.csv", index_col= "crn")
schedule_df.head()

,code,day,starts,ends
crn,,,,
13576,KIM 101E,Pazartesi,1130,1429
10019,KIM 101E,Pazartesi,1430,1729
10021,KIM 101E,Pazartesi,1430,1729
10033,KIM 101E,Pazartesi,1430,1729
10023,KIM 101E,Salı,830,1129


In [17]:
import ipywidgets as widgets

print("Select courses")
selection = widgets.SelectMultiple(
                    options=codes_full,
                    description='Courses',
                    disabled=False
                )
display(selection)

Select courses


SelectMultiple(description='Courses', options=('KIM 101E', 'KIM 101EL', 'BIL 101E', 'MAT 111', 'FIZ 101E', 'FI…

In [40]:
selection.value

('KIM 101E',
 'BIL 101E',
 'FIZ 106',
 'TUR 102',
 'ING 201',
 'ATA 101',
 'EKO 201',
 'ITB 171',
 'END 337',
 'MAT 365',
 'MAT 448')

In [41]:
selected_df = schedule_df.loc[schedule_df["code"].isin(selection.value)]
schedule_df.reset_index(inplace = True)

In [50]:
def fill_value(a, b):
    if(a.code != b.code):
        if (((a.day == b.day) and (a.starts > int(b.ends) or b.starts > int(a.ends))) or (a.day != b.day)):
            return 1
    else:
        return 0

In [51]:
arr_len = len(selected_df)
G = np.zeros([arr_len,arr_len])

for i in range(arr_len):
    for j in range(i, arr_len):
        G[i][j] = fill_value(schedule_df.iloc[i], schedule_df.iloc[j])
        G[j][i] = fill_value(schedule_df.iloc[i], schedule_df.iloc[j])

In [52]:
def pick_a_pivot(graph, vertices):
    maximum = 0
    vertex = ""
    for vertice in vertices:
        counter = 0
        for neighvertice in vertices:
            if(graph[vertice][neighvertice] == 1):
                counter = counter + 1 
        if(counter >= maximum):
            maximum = counter
            vertex = vertice
    return vertex
def N(G, vertex): 
    neighbours = []
    i = 0
    for i in range(len(G)):
        if(G[vertex][i] == 1):
            neighbours.append(i)
    return neighbours

def bronKerbosch2(R,P,X,G, results):
    if (len(P)==0 and len(X) ==0):
#         print(R)
        results.append(R)
        return        
    u = pick_a_pivot(G,list(set(P).union(set(X))))
    for v in list(set(P).difference(set(N(G, u)))):
        new_R = list(set(R).union({v}))
        new_P = list(set(P).intersection(set(N(G, v))))
        new_X = list(set(X).intersection(set(N(G,v))))
        bronKerbosch2(new_R, new_P, new_X, G, results)
        set_v = set()
        set_v.add(v)
        P = list((set(P).difference(set_v)))
        X = list((set(X).union(set_v)))

        
P = [i for i in range(arr_len)]
X = []   
counter = 0
results = []
bronKerbosch2([], P, [], G, results)

In [54]:
shapes = []
for result in results:
    shapes.append(len(result))
shapes = set(shapes)

print("Select the number of courses in your schedule")
shapes_selection = widgets.SelectMultiple(
                        options=shapes,
                        description='Number of courses',
                        disabled=False
                )
display(shapes_selection)

Select the number of courses in your schedule


SelectMultiple(description='Number of courses', options=(9, 10, 11, 12, 13, 14), value=())

In [55]:
possible_schedules = []
for value in shapes_selection.value:
    for result in results:
        if len(result) == value:
            possible_schedules.append([("crn: " + str(schedule_df.iloc[i].crn) + " code: " + str(schedule_df.iloc[i].code)) for i in result])

possible_schedules

[['crn: 10880 code: MAT 221',
  'crn: 10893 code: MAT 242',
  'crn: 10958 code: MAT 310',
  'crn: 10965 code: MAT 337E',
  'crn: 10039 code: KIM 101E',
  'crn: 10042 code: KIM 101EL',
  'crn: 11004 code: MAT 420E',
  'crn: 10370 code: FIZ 101E',
  'crn: 10388 code: FIZ 101EL',
  'crn: 10926 code: MUH 321'],
 ['crn: 10880 code: MAT 221',
  'crn: 10893 code: MAT 242',
  'crn: 10958 code: MAT 310',
  'crn: 10965 code: MAT 337E',
  'crn: 10039 code: KIM 101E',
  'crn: 10042 code: KIM 101EL',
  'crn: 11004 code: MAT 420E',
  'crn: 10732 code: FIZ 101E',
  'crn: 10388 code: FIZ 101EL',
  'crn: 10926 code: MUH 321'],
 ['crn: 10019 code: KIM 101E',
  'crn: 10880 code: MAT 221',
  'crn: 10893 code: MAT 242',
  'crn: 10958 code: MAT 310',
  'crn: 10965 code: MAT 337E',
  'crn: 10042 code: KIM 101EL',
  'crn: 11004 code: MAT 420E',
  'crn: 10388 code: FIZ 101EL',
  'crn: 10460 code: FIZ 101E',
  'crn: 10926 code: MUH 321'],
 ['crn: 10021 code: KIM 101E',
  'crn: 10880 code: MAT 221',
  'crn: 1089